In [1]:
## Dwa punkty, przestrzeń 2D
## Niech E - punkt startowy uciekającego, P - punkt startowy goniącego
## E porusza się po lini prostej, z prędkością v = const
## P porusza się w kierunku E, z prędkością w = const. Tor P to krzywa pogoni.

## v = (vx, vy)
## w = (E_x-P_x, E_y-P_y)
## Po czasie dt, położenie E będzie
## d(E, dt) = (E_x + vxdt, E_y + vydt)
## Wtedy zaktualizowany wektor prędkości P po upływie czasu dt to
## w = (E_x+vxdt-P_x, E_y+vydt-P_y)



In [35]:
from __future__ import annotations
import math
from dataclasses import dataclass

@dataclass
class Point2D:
    x: float
    y: float

    def __add__(self, val: Point2D) -> Point2D:
        return Point2D(self.x+val.x, self.y+val.y)

    def __sub__(self, val: Point2D) -> Point2D:
        return Point2D(self.x-val.x, self.y-val.y)


class DirectPursuit:
    """Kierunek wprost na cel."""
    def calculate_movement(self, pursuer: Point2D, target: Point2D, pursuer_velocity: Point2D) -> Point2D:
        w = target - pursuer
        l = (w.x*w.x + w.y*w.y) ** 0.5
        return Point2D(pursuer_velocity.x*w.x/l,
                       pursuer_velocity.y*w.y/l)

class ConstantBearing:
    """Kierunek do celu, ale odchylony o dany kąt."""
    def __init__(self, bearing_angle_deg: float):
        self.bearing_angle = math.radians(bearing_angle_deg)
    
    def calculate_movement(self, pursuer: Point2D, target: Point2D, pursuer_velocity: Point2D) -> Point2D:
        w = target - pursuer        
        angle_to_target = math.atan2(w.y, w.x)
        movement_angle = angle_to_target + self.bearing_angle
        return Point2D(pursuer_velocity.x * math.cos(movement_angle), 
                pursuer_velocity.y * math.sin(movement_angle))

class ProportionalNavigation:
    """
    Używana w rakietach i pociskach. 
    Prędkość kątowa ścigającego jest proporcjonalna do prędkości kątowej linii namiarowania (LOS).
    https://en.wikipedia.org/wiki/Proportional_navigation
    """

    def __init__(self, N: float = 3.0):
        self.N = N
        self.previous_los_angle: float | None = None
        self.previous_pursuer_vel: Point2D | None = None

    def calculate_movement(self, pursuer: Point2D, target: Point2D, pursuer_velocity: Point2D) -> Point2D:
        w = target - pursuer
        dist = (w.x*w.x + w.y+w.y) ** 0.5

        los_angle = math.atan2(w.y, w.x)
        if self.previous_los_angle is None:
            self.previous_los_angle = los_angle
            self.previous_pursuer_vel = Point2D(
                pursuer_velocity.x * math.cos(los_angle),
                pursuer_velocity.y * math.sin(los_angle),
            )
            return self.previous_pursuer_vel
            
        delta_los_angle = los_angle - self.previous_los_angle
        while delta_los_angle > math.pi:  delta_los_angle -= 2 * math.pi
        while delta_los_angle < -math.pi: delta_los_angle += 2 * math.pi

        if self.previous_pursuer_vel is None:
            pursuer_angle = los_angle
        else:
            pursuer_angle = math.atan2(self.previous_pursuer_vel.y, self.previous_pursuer_vel.x)

        new_pursuer_angle = pursuer_angle + self.N * delta_los_angle
        new_vel = Point2D(
            pursuer_velocity.x * math.cos(new_pursuer_angle),
            pursuer_velocity.y * math.sin(new_pursuer_angle),
        )

        self.previous_los_angle = los_angle
        self.previous_pursuer_vel = new_vel
        return new_vel


class ParallelNavigation:
    """Przewiduje pozycje celu. Troche ułomna, można wywalić"""
    def __init__(self):
        self.previous_target_pos: Point2D | None = None
    
    def calculate_movement(self, pursuer_pos: Point2D, target_pos: Point2D, pursuer_velocity: Point2D) -> Point2D:
        # Estymacja prędkości celu
        if self.previous_target_pos is None:
            target_velocity = Point2D(0, 0)
        else:
            target_velocity = target_pos - self.previous_target_pos
        
        self.previous_target_pos = target_pos
        
        # Przewidywana pozycja celu (prosty predyktor)
        prediction_steps = 5
        predicted_pos = Point2D(target_pos.x + target_velocity.x*prediction_steps,
                                target_pos.y + target_velocity.y*prediction_steps)
        
        w = predicted_pos - pursuer_pos
        l = (w.x*w.x + w.y*w.y)**0.3
        
        return Point2D(pursuer_velocity.x * w.x/l, pursuer_velocity.y * w.y/l)

class Simulation:
    def __init__(self, pursuer_start: Point2D, target_start: Point2D, pursuer_velocity: Point2D, target_velocity: Point2D, strategy, max_iters: int = 1000):
        self.pursuer_positions = [pursuer_start]
        self.target_positions = [target_start]
        self.pursuer_velocity = pursuer_velocity
        self.target_velocity = target_velocity
        self.strategy = strategy
        self.max_iters = max_iters

    def _step(self) -> tuple[Point2D, Point2D]:
        target = self.target_positions[-1]
        target += self.target_velocity
        self.target_positions.append(target)

        pursuer = self.pursuer_positions[-1]
        movement = self.strategy.calculate_movement(pursuer, target, self.pursuer_velocity)
        pursuer += movement
        self.pursuer_positions.append(pursuer)

        return pursuer, target

    def _should_stop(self, pursuer: Point2D, target: Point2D) -> bool:
        dx = target.x - pursuer.x
        dy = target.y - pursuer.y
        dist = (dx*dx + dy*dy) ** 0.5
        return dist < 0.45
    
    def run(self) -> None:
        caught = False
        for i in range(self.max_iters):
            t, p = self._step()
            if self._should_stop(p, t):
                print(f"Złapano cel po {i} krokach.")
                caught = True
                break
        if not caught:
            print(f"Nie udało się złapać celu po {self.max_iters} krokach.")

In [36]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

def animate_pursuit(simulation):
    tps = simulation.target_positions
    pps = simulation.pursuer_positions
    
    fig, ax = plt.subplots(figsize=(8, 8))
    
    all_x = [point.x for point in tps] + [point.x for point in pps]
    all_y = [point.y for point in tps] + [point.y for point in pps]
    ax.set_xlim(min(all_x) - 1, max(all_x) + 1)
    ax.set_ylim(min(all_y) - 1, max(all_y) + 1)
    
    line_target, = ax.plot([], [], 'ro-', label='Target (Cel)', 
                           linewidth=2, markersize=8)
    line_pursuer, = ax.plot([], [], 'bo-', label='Pursuer (Ścigający)', 
                            linewidth=2, markersize=8)
    
    current_target, = ax.plot([], [], 'ro', markersize=15, alpha=0.7)
    current_pursuer, = ax.plot([], [], 'bo', markersize=15, alpha=0.7)
    
    ax.legend()
    ax.grid(True)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    
    def init():
        line_target.set_data([], [])
        line_pursuer.set_data([], [])
        current_target.set_data([], [])
        current_pursuer.set_data([], [])
        return line_target, line_pursuer, current_target, current_pursuer
    
    def animate(frame):
        x_coords_target = [tps[i].x for i in range(frame+1)]
        y_coords_target = [tps[i].y for i in range(frame+1)]
        line_target.set_data(x_coords_target, y_coords_target)
        
        x_coords_pursuer = [pps[i].x for i in range(frame + 1)]
        y_coords_pursuer = [pps[i].y for i in range(frame + 1)]
        line_pursuer.set_data(x_coords_pursuer, y_coords_pursuer)
        
        current_target.set_data([tps[frame].x], [tps[frame].y])
        current_pursuer.set_data([pps[frame].x], [pps[frame].y])
        
        return line_target, line_pursuer, current_target, current_pursuer
    
    anim = animation.FuncAnimation(fig, animate, init_func=init,
                                 frames=len(tps), interval=50, 
                                 blit=True, repeat=True)
    plt.close()
    return HTML(anim.to_jshtml())

sim = Simulation(
    pursuer_start=Point2D(0.0, 0.0),
    target_start=Point2D(10.0, 3.0),
    pursuer_velocity=Point2D(2.2, 1.8),
    target_velocity=Point2D(1.5, -0.5),
    strategy=ProportionalNavigation(N=3.0),
    max_iters=100
)
sim.run()
animate_pursuit(sim)

Złapano cel po 14 krokach.
